In [10]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from collections import defaultdict
import random
import csv

In [11]:
# Define the parameters

# Path to the CSV file containing the cost data
data_path = "./cost_10_07.csv"

# Column name in the CSV file that represents the part
part_col = "Part"

# Column name in the CSV file that represents the departure/arrival site
dep_arr_site_col = "Departure/Arrival site"

# Column name in the CSV file that represents the arrival/departure site
arr_dep_site_col =  "Arrival/Departure site"

# Column name in the CSV file that represents the cost
cost_col = "Cost"

# List of tuples representing the part and its corresponding site
# Each tuple is in the format (part, site)
PBS = [(2, 1), (3, 1), (4, 1), (5, 2) , (6, 2), (7, 2), (8, 3), (9, 4), (10, 4)]

# List of tuples representing the subparts
# Each tuple is in the format (subpart1, subpart2)
subpart = [(2,3),(2,4),(3,4),(5,6),(5,7),(6,7),(9,10)]

# Penalty parameters for the optimization problem
lambda_1 = 10
lambda_2 = 10
lambda_3 = 10

In [12]:
# Read data from the CSV file
data = pd.read_csv(data_path)
# Store it in a pandas dataframe
data_df = pd.DataFrame(data)

In [13]:
# Define the function to create the cost matrix DataFrame
def cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col):

    # Define the range of parts and sites
    # The range of parts starts from 2 and goes up to the maximum part number
    # The range of sites starts from 1 and goes up to the maximum site number
    part_range = range(1, data_df[part_col].max() + 1)
    site_range = range(1, max(data_df[dep_arr_site_col].max(), data_df[arr_dep_site_col].max()) + 1)

    pivot_df = (
        pd.DataFrame(
            index=pd.MultiIndex.from_product(
                [part_range, site_range, site_range], 
                names=['Part', 'Departure_Site', 'Arrival_Site']
            ) # Create a pivot DataFrame with a multi-index of parts, departure sites, and arrival sites
        )
        .reset_index() # Rest index of the dataframe
        .merge(data_df.assign(Part_Column=data_df[part_col]), how='left'
               , left_on=['Part', 'Departure_Site', 'Arrival_Site'] # Merge the pivot DataFrame with the original DataFrame on the part, departure site, and arrival site columns
               , right_on=[part_col, dep_arr_site_col, arr_dep_site_col])
        .assign(Cost=lambda df: df[cost_col].fillna(0)) # Fill the missing cost values with 0
        .pivot_table(index=['Part', 'Departure_Site']
                     , columns=['Part_Column', 'Arrival_Site']
                     , values='Cost'
                     , fill_value=0) # Pivot the DataFrame to create a cost matrix with the part and departure site as the index and the part and arrival site as the columns
        .pipe(lambda df: df.reorder_levels(['Part_Column', 'Arrival_Site'], axis=1))    # Reorder the levels of the columns
        .pipe(lambda df: df.reindex(columns=pd.MultiIndex.from_product([part_range, site_range]
                                                                       , names=df.columns.names)
                                    , fill_value=0)) # Reindex the columns and the index to include all combinations of parts and sites
        .pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([part_range, site_range], names=df.index.names), fill_value=0))
        .pipe(lambda df: df + df.T) # Make the DataFrame symmetric by adding its transpose to itself
        .pipe(lambda df: df.where(np.eye(df.shape[0], dtype=bool), df / 2)) # Halve the off-diagonal elements and keep the diagonal the same
    )

    # Return the pivot DataFrame
    return pivot_df

In [14]:
cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)

Part_Column           1                                  2             ...  \
Arrival_Site           1    2    3    4    5    6    7    1    2    3  ...   
Part Departure_Site                                                    ...   
1    1               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     2               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     3               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     4               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     5               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
10   3               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     4               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     5               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     6               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
     7               0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

Part_Column           9                10                                    \
Arrival_Site           5    6    7      1      2     3      4      5      6   
Part Departure_Site                                                           
1    1               0.0  0.0  0.0  0.000  0.000  0.00  0.000  0.000  0.000   
     2               0.0  0.0  0.0  0.000  0.000  0.00  0.000  0.000  0.000   
     3               0.0  0.0  0.0  0.000  0.000  0.00  0.000  0.000  0.000   
     4               0.0  0.0  0.0  0.000  0.000  0.00  0.000  0.000  0.000   
     5               0.0  0.0  0.0  0.000  0.000  0.00  0.000  0.000  0.000   
...                  ...  ...  ...    ...    ...   ...    ...    ...    ...   
10   3               0.0  0.0  0.0  1.560  0.875  0.00  1.390  1.510  2.010   
     4               0.0  0.0  0.0  1.625  0.545  1.39  0.000  2.465  0.285   
     5               0.0  0.0  0.0  2.135  1.670  1.51  2.465  0.000  1.550   
     6               0.0  0.0  0.0  1.360  0.360  2.01  0.285  1.550  0.000   
     7               0.0  0.0  0.0  2.525  2.660  2.45  2.260  1.035  2.335   

Part_Column                 
Arrival_Site             7  
Part Departure_Site         
1    1               0.000  
     2               0.000  
     3               0.000  
     4               0.000  
     5               0.000  
...                    ...  
10   3               2.450  
     4               2.260  
     5               1.035  
     6               2.335  
     7               0.000  

[70 rows x 70 columns]

In [15]:
def cost_function_qubo(PBS):
    pivot_df = cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)
    site_range = pivot_df.index.get_level_values('Departure_Site').unique()

    # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    cost_function = {((r, i), (r, j)): pivot_df.loc[(r, i), (r, j)] 
            for r in range(1, max_part + 1) for i in site_range for j in site_range if i != j and (r, i) in pivot_df.index and (r, j) in pivot_df.index}

    return cost_function

In [16]:
def constraint_C1(PBS):
    
    
    # Initialize the constraint dictionary
    constraint_C1 = {}
    
    max_sites = data_df['Arrival/Departure site'].max()
    # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    # Define the range of sites
    site_range = range(1, max_sites + 1)  # Sites go from 1 to num_sites

    # Loop over all parts and sites to build the QUBO terms
    for part in range(1, max_part + 1):
        for i in site_range:
            for j in site_range:
                if i == j:
                    # Diagonal terms: -2 * x_{part i}^2
                    constraint_C1[(part, i), (part, i)] = -2
                else:
                    # Off-diagonal terms: x_{part i} * x_{part j}
                    constraint_C1[(part, i), (part, j)] = 1

    return constraint_C1

In [17]:
constraint_C1(PBS)

{((1, 1), (1, 1)): -2,
 ((1, 1), (1, 2)): 1,
 ((1, 1), (1, 3)): 1,
 ((1, 1), (1, 4)): 1,
 ((1, 1), (1, 5)): 1,
 ((1, 1), (1, 6)): 1,
 ((1, 1), (1, 7)): 1,
 ((1, 2), (1, 1)): 1,
 ((1, 2), (1, 2)): -2,
 ((1, 2), (1, 3)): 1,
 ((1, 2), (1, 4)): 1,
 ((1, 2), (1, 5)): 1,
 ((1, 2), (1, 6)): 1,
 ((1, 2), (1, 7)): 1,
 ((1, 3), (1, 1)): 1,
 ((1, 3), (1, 2)): 1,
 ((1, 3), (1, 3)): -2,
 ((1, 3), (1, 4)): 1,
 ((1, 3), (1, 5)): 1,
 ((1, 3), (1, 6)): 1,
 ((1, 3), (1, 7)): 1,
 ((1, 4), (1, 1)): 1,
 ((1, 4), (1, 2)): 1,
 ((1, 4), (1, 3)): 1,
 ((1, 4), (1, 4)): -2,
 ((1, 4), (1, 5)): 1,
 ((1, 4), (1, 6)): 1,
 ((1, 4), (1, 7)): 1,
 ((1, 5), (1, 1)): 1,
 ((1, 5), (1, 2)): 1,
 ((1, 5), (1, 3)): 1,
 ((1, 5), (1, 4)): 1,
 ((1, 5), (1, 5)): -2,
 ((1, 5), (1, 6)): 1,
 ((1, 5), (1, 7)): 1,
 ((1, 6), (1, 1)): 1,
 ((1, 6), (1, 2)): 1,
 ((1, 6), (1, 3)): 1,
 ((1, 6), (1, 4)): 1,
 ((1, 6), (1, 5)): 1,
 ((1, 6), (1, 6)): -2,
 ((1, 6), (1, 7)): 1,
 ((1, 7), (1, 1)): 1,
 ((1, 7), (1, 2)): 1,
 ((1, 7), (1, 3)): 1,
 ((1

In [18]:
len(constraint_C1(PBS))

490

In [19]:
def constraint_C2(PBS):

    # Define the range of sites
    num_sites = data_df['Arrival/Departure site'].max()
     # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    # Initialize the constraint dictionary
    constraint_C2 = {}
    
    # Loop over all parts part in the PBS and sites i to build the QUBO terms
    for part in range(1, max_part + 1):
        for i in range(1, num_sites + 1):
            # Since r and s are the same, we only need to consider the terms x_{ri} x_{ri}
            # Add a penalty term for assigning part r to the same site as both origin and destination
            constraint_C2[(part, i), (part, i)] = 1  # The penalty weight is set to 1

    return constraint_C2

In [60]:
def constraint_C3(subpart):
    # Initialize the constraint dictionary
    constraint_C3 = {}
    num_sites = data_df['Arrival/Departure site'].max()


    # Loop over all tuples in subparts and sites to build the QUBO terms
    for r, s in subpart:
        for i in range(1, num_sites + 1):
            # Add a penalty term for assigning both sub-parts r and s to the same site i
             constraint_C3[(r, i), (s, i)] = 1  # The penalty weight is set to 1

    return constraint_C3

In [63]:
constraint_C3(subpart)

{((2, 1), (3, 1)): 1,
 ((2, 2), (3, 2)): 1,
 ((2, 3), (3, 3)): 1,
 ((2, 4), (3, 4)): 1,
 ((2, 5), (3, 5)): 1,
 ((2, 6), (3, 6)): 1,
 ((2, 7), (3, 7)): 1,
 ((2, 1), (4, 1)): 1,
 ((2, 2), (4, 2)): 1,
 ((2, 3), (4, 3)): 1,
 ((2, 4), (4, 4)): 1,
 ((2, 5), (4, 5)): 1,
 ((2, 6), (4, 6)): 1,
 ((2, 7), (4, 7)): 1,
 ((3, 1), (4, 1)): 1,
 ((3, 2), (4, 2)): 1,
 ((3, 3), (4, 3)): 1,
 ((3, 4), (4, 4)): 1,
 ((3, 5), (4, 5)): 1,
 ((3, 6), (4, 6)): 1,
 ((3, 7), (4, 7)): 1,
 ((5, 1), (6, 1)): 1,
 ((5, 2), (6, 2)): 1,
 ((5, 3), (6, 3)): 1,
 ((5, 4), (6, 4)): 1,
 ((5, 5), (6, 5)): 1,
 ((5, 6), (6, 6)): 1,
 ((5, 7), (6, 7)): 1,
 ((5, 1), (7, 1)): 1,
 ((5, 2), (7, 2)): 1,
 ((5, 3), (7, 3)): 1,
 ((5, 4), (7, 4)): 1,
 ((5, 5), (7, 5)): 1,
 ((5, 6), (7, 6)): 1,
 ((5, 7), (7, 7)): 1,
 ((6, 1), (7, 1)): 1,
 ((6, 2), (7, 2)): 1,
 ((6, 3), (7, 3)): 1,
 ((6, 4), (7, 4)): 1,
 ((6, 5), (7, 5)): 1,
 ((6, 6), (7, 6)): 1,
 ((6, 7), (7, 7)): 1,
 ((9, 1), (10, 1)): 1,
 ((9, 2), (10, 2)): 1,
 ((9, 3), (10, 3)): 1,
 ((9, 4

In [64]:
def constraint_C3(subparts):
    # Define the range of sites
    num_sites = data_df['Arrival/Departure site'].max()
     # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)
    
    # Initialize the constraint dictionary
    constraint_C3 = {}

    # Loop over all tuples in subparts and sites to build the QUBO terms
    for r, s in subparts:
        for i in range(1, num_sites + 1):
            # Add a penalty term for assigning both sub-parts r and s to the same site i
            constraint_C3[(r, i), (s, i)] = 1  # The penalty weight is set to 1

    return constraint_C3

In [65]:
# Auxiliary function to multiply penalties by QUBO dictionaries

def multiply_penalty_qubo(penalty, qubo_dict):
    # Create a new QUBO dictionary to hold the result
    result_qubo = {}

    # Multiply each value in the QUBO dictionary by the penalty
    for key, value in qubo_dict.items():
        result_qubo[key] = penalty * value

    return result_qubo

In [66]:
# Auxiliary function to add QUBO dictionaries
def add_qubo_dicts(qubo_dict1, qubo_dict2):
    # Create a new defaultdict to hold the result
    result_qubo = defaultdict(int)

    # Add the values from the first QUBO dictionary
    for key, value in qubo_dict1.items():
        result_qubo[key] += value

    # Add the values from the second QUBO dictionary
    for key, value in qubo_dict2.items():
        result_qubo[key] += value

    return dict(result_qubo)

In [67]:
def calculate_qubo(penalties_qubos):
    # Initialize Q as an empty dictionary
    Q = {}
    # Iterate over the list of tuples
    for penalty, qubo in penalties_qubos:
        # Multiply the QUBO dictionary by its penalty
        qubo_multiplied = multiply_penalty_qubo(penalty, qubo)
        
        # Add the multiplied QUBO dictionary to Q
        Q = add_qubo_dicts(Q, qubo_multiplied)
    
    return Q

In [68]:
def qubo_matrix_df(Q):
    Q_df = (
        pd.Series(Q)  # Convert the QUBO dictionary into a pandas Series
        .pipe(lambda r: pd.DataFrame(r, index=pd.MultiIndex.from_tuples(r.index)))  # Create a DataFrame with a multi-index from the keys of the QUBO dictionary
        .unstack(level=0)  # Unstack the DataFrame to create a matrix
        .transpose()  # Transpose the DataFrame
        .fillna(0)  # Fill NaN values with 0
        .pipe(lambda df: df if df.index.name is None else df.rename_axis(None))  # Remove the index name
        .pipe(lambda df: df.set_index(df.index.map(lambda x: str(x[1]) if isinstance(x, tuple) else str(x))))  # Convert the multi-index into a string representation of the tuple
    )

    return Q_df

In [69]:
# C is the QUBO matrix for the cost function.
# The cost_function_qubo function takes PBS as an argument and returns
# the QUBO matrix.
C = cost_function_qubo(PBS)

# C1 is the QUBO matrix for the first penalty term.
# The penalty_C1_qubo function takes PBS as an argument and returns
# the QUBO matrix for the first penalty term.
C1 = constraint_C1(PBS)

# C2 is the QUBO matrix for the second penalty term.
# The penalty_C2_qubo function takes PBS as an argument and returns
# the QUBO matrix for the second penalty term.
C2 = constraint_C2(PBS)

# C3 is the QUBO matrix for the third penalty term.
# The penalty_C3_qubo function takes subpart as an argument and returns
# the QUBO matrix for the third penalty term.
C3 = constraint_C3(subpart)

In [76]:
# Introduce the structure of the Qubo problem
# (1, C), (1, C1), (1, C2), (1, C3)
penalties_qubos = [(1, C), (1, C1), (1, C2), (1, C3)]
# Calculate the Qubo matrix
Q = calculate_qubo(penalties_qubos)
# QUBO matrix as a df for inspection
Q_df = qubo_matrix_df(Q)
# Show Q_df
Q_df

,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)","(1, 6)","(1, 7)","(2, 1)","(2, 2)","(2, 3)","(2, 4)","(2, 5)","(2, 6)","(2, 7)","(3, 1)","(3, 2)","(3, 3)","(3, 4)","(3, 5)","(3, 6)","(3, 7)","(4, 1)","(4, 2)","(4, 3)","(4, 4)","(4, 5)","(4, 6)","(4, 7)","(5, 1)","(5, 2)","(5, 3)","(5, 4)","(5, 5)","(5, 6)","(5, 7)","(6, 1)","(6, 2)","(6, 3)","(6, 4)","(6, 5)","(6, 6)","(6, 7)","(7, 1)","(7, 2)","(7, 3)","(7, 4)","(7, 5)","(7, 6)","(7, 7)","(8, 1)","(8, 2)","(8, 3)","(8, 4)","(8, 5)","(8, 6)","(8, 7)","(9, 1)","(9, 2)","(9, 3)","(9, 4)","(9, 5)","(9, 6)","(9, 7)","(10, 1)","(10, 2)","(10, 3)","(10, 4)","(10, 5)","(10, 6)","(10, 7)"
"(1, 1)",-1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 2)",1.0,-1.0,1.0,1.0,1.0,1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 3)",1.0,1.0,-1.0,1.0,1.0,1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 4)",1.0,1.0,1.0,-1.0,1.0,1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 5)",1.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 6)",1.0,1.0,1.0,1.0,1.0,-1.0,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(1, 7)",1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(2, 1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000,1.820,1.525,1.545,1.715,1.455,1.850,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"(2, 2)",0.0,0.0,0.0,0.0,0.0,0.0,0

In [77]:
# Assuming df is your DataFrame

# Set the display options
pd.set_option('display.max_columns', None)  # No limit on columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to next line
pd.set_option('display.max_rows', None)  # No limit on rows

# Print the DataFrame
print(Q_df)

         (1, 1)  (1, 2)  (1, 3)  (1, 4)  (1, 5)  (1, 6)  (1, 7)  (2, 1)  (2, 2)  (2, 3)  (2, 4)  (2, 5)  (2, 6)  (2, 7)  (3, 1)  (3, 2)  (3, 3)  (3, 4)  (3, 5)  (3, 6)  (3, 7)  (4, 1)  (4, 2)  (4, 3)  (4, 4)  (4, 5)  (4, 6)  (4, 7)  (5, 1)  (5, 2)  (5, 3)  (5, 4)  (5, 5)  (5, 6)  (5, 7)  (6, 1)  (6, 2)  (6, 3)  (6, 4)  (6, 5)  (6, 6)  (6, 7)  (7, 1)  (7, 2)  (7, 3)  (7, 4)  (7, 5)  (7, 6)  (7, 7)  (8, 1)  (8, 2)  (8, 3)  (8, 4)  (8, 5)  (8, 6)  (8, 7)  (9, 1)  (9, 2)  (9, 3)  (9, 4)  (9, 5)  (9, 6)  (9, 7)  (10, 1)  (10, 2)  (10, 3)  (10, 4)  (10, 5)  (10, 6)  (10, 7)
(1, 1)     -1.0     1.0     1.0     1.0     1.0     1.0     1.0   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000   0.000  